# Optuna Study Visualization

Visualize hyperparameter optimization results from Optuna studies.

In [ ]:
import optuna
import optuna.visualization as vis
import mlflow

print(f"Optuna version: {optuna.__version__}")

In [ ]:
# Load study from SQLite (if persisted)
# study = optuna.load_study(study_name="xgboost-tuning", storage="sqlite:///optuna.db")

# Or create a quick demo study
import xgboost as xgb
import math
from sklearn.metrics import mean_squared_error
from scripts.generate_apple_sales_data import generate_apple_sales_data
from sklearn.model_selection import train_test_split

# Generate data
df = generate_apple_sales_data(n_rows=2000)
X = df.drop(columns=["date", "demand"])
y = df["demand"]
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25, random_state=42)
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(valid_x, label=valid_y)

def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "max_depth": trial.suggest_int("max_depth", 1, 9),
        "eta": trial.suggest_float("eta", 1e-3, 0.3, log=True),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    }
    model = xgb.train(params, dtrain, num_boost_round=50)
    preds = model.predict(dvalid)
    return mean_squared_error(valid_y, preds)

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print(f"Best MSE: {study.best_value:.4f}")
print(f"Best RMSE: {math.sqrt(study.best_value):.4f}")

## Optimization History

In [ ]:
fig = vis.plot_optimization_history(study)
fig.show()

## Parameter Importances

In [ ]:
fig = vis.plot_param_importances(study)
fig.show()

## Parallel Coordinate Plot

In [ ]:
fig = vis.plot_parallel_coordinate(study)
fig.show()

## Slice Plot

In [ ]:
fig = vis.plot_slice(study)
fig.show()

## Contour Plot

In [ ]:
fig = vis.plot_contour(study, params=["max_depth", "eta"])
fig.show()

## Best Parameters

In [ ]:
print("Best parameters:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")